In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
merged_df = pd.read_csv("clubbed_dataset.csv", parse_dates=['order_date'])


C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\264278398.py:8: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged_df = pd.read_csv("clubbed_dataset.csv", parse_dates=['order_date'])


In [ ]:
merged_df['item_sku_id'] = merged_df['item_name'].astype('category').cat.codes
sku_id_to_name = dict(enumerate(merged_df['item_name'].astype('category').cat.categories))
merged_df['quantity'].fillna(0, inplace=True)
merged_df['ordered'] = (merged_df['quantity'] > 0).astype(int)
merged_df['repeat_order_count'] = merged_df.groupby(['customer_id', 'item_sku_id']).cumcount()
merged_df['discount_per_unit'] = (merged_df['amount_x'] - merged_df['amount_y']) / merged_df['quantity']
merged_df['discount_per_unit'].replace([np.inf, -np.inf], 0, inplace=True)
merged_df['discount_per_unit'].fillna(0, inplace=True)
merged_df = merged_df.sort_values(['customer_id', 'item_sku_id', 'order_date'])
merged_df['qty_last7'] = merged_df.groupby(['customer_id', 'item_sku_id'])['quantity'].transform(lambda x: x.rolling(7, min_periods=1).mean())
merged_df['qty_last14'] = merged_df.groupby(['customer_id', 'item_sku_id'])['quantity'].transform(lambda x: x.rolling(14, min_periods=1).mean())
merged_df['order_count_last14'] = merged_df.groupby(['customer_id', 'item_sku_id'])['ordered'].transform(lambda x: x.rolling(14, min_periods=1).sum())
def compute_days_since_last_order(group):
    last_ordered = group[::-1].cumsum()[::-1]
    days_since = (~group.astype(bool)).astype(int).cumsum()
    days_since[last_ordered == 0] = 99
    return days_since

merged_df['days_since_last_order'] = merged_df.groupby(['customer_id', 'item_sku_id'])['ordered'].transform(compute_days_since_last_order)


C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\3887997918.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['quantity'].fillna(0, inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\3887997918.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [ ]:

cutoff_date = merged_df['order_date'].max() - pd.Timedelta(days=14)
train = merged_df[merged_df['order_date'] < cutoff_date]
test = merged_df[merged_df['order_date'] >= cutoff_date]

features = ['qty_last7', 'qty_last14', 'order_count_last14', 'days_since_last_order', 'repeat_order_count', 'discount_per_unit']

X_train = train[features]
y_train = train['ordered']
X_test = test[features]

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

if 1 in clf.classes_:
    test['prob_order'] = clf.predict_proba(X_test)[:, list(clf.classes_).index(1)]
else:
    test['prob_order'] = 0
test['item_name'] = test['item_sku_id'].map(sku_id_to_name)

test[['customer_id', 'item_name', 'order_date', 'prob_order']].to_csv('sku_level_product_order_prediction.csv', index=False)
print("✅ SKU-level order predictions saved to 'sku_level_product_order_prediction.csv'")


✅ SKU-level order predictions saved to 'sku_level_product_order_prediction.csv'


C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\956895533.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['prob_order'] = clf.predict_proba(X_test)[:, list(clf.classes_).index(1)]
C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\956895533.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['item_name'] = test['item_sku_id'].map(sku_id_to_name)
